In [ ]:
!pip install transformers[torch] datasets

In [ ]:
from datasets import load_dataset
squad = load_dataset("squad")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
record = squad["train"][0]
display(record.keys())

# Output
# dict_keys(['id', 'title', 'context', 'question', 'answers'])

display(record)

dict_keys(['id', 'title', 'context', 'question', 'answers'])

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# test tokenizer
tokenizer("Hello, my dog is cute", return_tensors="pt")

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# Alguns exemplos em um conjunto de dados podem ter um contexto muito longo que excede o comprimento máximo de entrada do modelo.
# Para lidar com sequências mais longas, trunque apenas o contexto definindo truncation="only_second"
# Em seguida, mapeie as posições inicial e final da resposta para o contexto original definindo return_offset_mapping=True.
# Com o mapeamento em mãos, agora você pode encontrar os tokens inicial e final da resposta.
# Use o método sequence_ids para descobrir qual parte do deslocamento corresponde à pergunta e qual corresponde ao contexto

def preprocess_function(examples):
    """Preprocesses the data for the model.

    Args:
        examples (dict): The examples to be preprocessed.

    Returns:
        dict: The preprocessed examples.
    """
    questions = [q.strip() for q in examples["question"]] # getting question argument

    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second", # truncate context that exeeds max input length of the model
        return_offsets_mapping=True,# map start and end positions of the answer to origninal context
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_squad = squad.map(preprocess_function,
                            batched=True,
                            remove_columns=squad["train"].column_names)

print(tokenized_squad)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10570
    })
})


In [ ]:
# Load HuggingFace's default data collator class
from transformers import default_data_collator
data_collator = default_data_collator

# instantiate the model
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='./sample_data/results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.292600,1.188803


TrainOutput(global_step=5475, training_loss=1.5513528289098173, metrics={'train_runtime': 3467.24, 'train_samples_per_second': 25.265, 'train_steps_per_second': 1.579, 'total_flos': 8583810682277376.0, 'train_loss': 1.5513528289098173, 'epoch': 1.0})

In [ ]:
trainer.save_model('./sample_data/results')

In [ ]:
tokenizer.save_pretrained('./sample_data/results')

('./sample_data/results/tokenizer_config.json',
 './sample_data/results/special_tokens_map.json',
 './sample_data/results/vocab.txt',
 './sample_data/results/added_tokens.json',
 './sample_data/results/tokenizer.json')

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Caminho para o diretório onde o modelo foi salvo
model_path = "./sample_data/results"

# Carregue o modelo e o tokenizador
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Exemplo de pergunta
question = "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?"
context = "Architecturally, the school has a Catholic character. Atop the Main Buildings gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend Venite Ad Me Omnes Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary."

# Tokenize a pergunta e o contexto
inputs = tokenizer(question, context, return_tensors="pt")

# Faça a predição
outputs = model(**inputs)
start_scores = outputs.start_logits
end_scores = outputs.end_logits

# Encontre o início e o fim da resposta
start_index = start_scores.argmax(1).item()
end_index = end_scores.argmax(1).item()

# Obtenha a resposta a partir dos índices
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index + 1]))

# Imprima a resposta
print(answer)


saint bernadette soubirous


In [ ]:
from transformers import pipeline

# Carregue o modelo treinado
model_path = "./sample_data/results"  # Este deve ser o mesmo caminho que você especificou em output_dir
qa_pipeline = pipeline(
    task="question-answering",
    model=model_path,
    tokenizer=model_path,
)

# Exemplo de pergunta
question = "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?"
context = "Architecturally, the school has a Catholic character. Atop the Main Buildings gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend Venite Ad Me Omnes Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary."

# Execute a inferência
answer = qa_pipeline(question=question, context=context)

# Imprima a resposta
print(answer)


{'score': 0.6687347888946533, 'start': 511, 'end': 537, 'answer': 'Saint Bernadette Soubirous'}
